In [30]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from GeneralTools.auxiliary_functions import reformat_labels
from GeneralTools.strings import FLAGS
import time
import os.path

In [31]:
# Load the dataset
fashion_mnist = keras.datasets.fashion_mnist
(train_images_raw, train_labels_raw), (test_images, test_labels) = fashion_mnist.load_data()
train_vis = train_images_raw
labels_vis = train_labels_raw

# flatten image data from num_samples x 28 x 28 to num_samples x 784
train_images_raw = train_images_raw.reshape((train_images_raw.shape[0], 784))
test_images = test_images.reshape((test_images.shape[0], 784))

# reformat label data to num_samples x 10
train_labels_raw = reformat_labels(train_labels_raw)
test_labels = reformat_labels(test_labels)

In [32]:
# prepare the dataset for batching
features_placeholder = tf.placeholder(tf.float32, train_images_raw.shape)
labels_placeholder = tf.placeholder(tf.int32, train_labels_raw.shape)
dataset_tr = tf.data.Dataset.from_tensor_slices(
    {"features": features_placeholder,
    "labels": labels_placeholder})

# scale the images
dataset_tr = dataset_tr.map(lambda d: (d['features'] / 255.0, d['labels']))

batch_size = 400
shuffle = True
buffer_size = 10000
num_epoch = None # set this to None or -1 will repeat the dataset infinite times

# shuffle
if shuffle:
    dataset_tr = dataset_tr.shuffle(buffer_size)
        
# make batch
dataset_tr = dataset_tr.batch(batch_size)
# repeat datasets for num_epoch
dataset_tr = dataset_tr.repeat(num_epoch)

# define an iterator that reads a batch each time
iterator = dataset_tr.make_initializable_iterator()
# read a batch
train_images, train_labels = iterator.get_next()

train_images = tf.cast(train_images, tf.float32)
test_images = tf.cast(test_images, tf.float32)

In [16]:
hidden_layer_nodes = 128
output_layer_nodes = 10

In [17]:
# set up NN (1 hidden layer, 1 output layer)
with tf.variable_scope('LinearModel0', reuse=tf.AUTO_REUSE):
    # now declare the weights connecting the input to the hidden layer of 10 nodes
    w0 = tf.get_variable('weight0', shape=(train_images.shape[1], hidden_layer_nodes), dtype=tf.float32,
                         initializer=tf.random_normal_initializer, trainable=True)
    b0 = tf.get_variable('bias0', shape=[hidden_layer_nodes], dtype=tf.float32, initializer=tf.random_normal_initializer, trainable=True)
    # and the weights connecting the hidden layer to the output layer
    w1 = tf.get_variable('weight1', shape=(hidden_layer_nodes, output_layer_nodes), dtype=tf.float32,
                         initializer=tf.random_normal_initializer, trainable=True)
    b1 = tf.get_variable('bias1', shape=[output_layer_nodes], dtype=tf.float32, initializer=tf.random_normal_initializer, trainable=True)

    # calculate the output of the hidden layer for training
    train_hidden_out = tf.add(tf.matmul(train_images, w0), b0)
    train_hidden_out = tf.nn.relu(train_hidden_out)

    # now calculate NN output - in this case, use a softmax activated output layer for training
    train_output_logits = tf.add(tf.matmul(train_hidden_out, w1), b1)
    train_output = tf.nn.softmax(train_output_logits)

    # calculate the output of the hidden layer for testing
    test_hidden_out = tf.add(tf.matmul(test_images, w0), b0)
    test_hidden_out = tf.nn.relu(test_hidden_out)

    # now calculate NN output - in this case, use a softmax activated output layer for testing
    test_output_logits = tf.add(tf.matmul(test_hidden_out, w1), b1)
    test_output = tf.nn.softmax(test_output_logits)

with tf.name_scope('LinearModel1'):
    pass

In [18]:
# define loss function
train_loss = tf.losses.softmax_cross_entropy(train_labels, train_output_logits, reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE, scope='train_loss')
test_loss = tf.losses.softmax_cross_entropy(test_labels, test_output_logits, reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE, scope='test_loss')

Instructions for updating:
Use tf.cast instead.


In [19]:
# determine accuracy
_,acc = tf.metrics.accuracy(test_output, test_labels)
acc = 100*acc

In [20]:
# configure the optimiser
global_step = tf.get_variable(name='global_step', shape=[], dtype=tf.int32, initializer=tf.constant_initializer(0), trainable=False)
init_learning_rate = 0.1
opt = tf.train.GradientDescentOptimizer(learning_rate=init_learning_rate, name='GD')

variable_list = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=None)
images_list = opt.compute_gradients(train_loss, variable_list)
opt_op = opt.apply_gradients(images_list, global_step=global_step)

In [21]:
# configure the summary
# summary op is always pinned to CPU
tf.summary.histogram('Input_Weights', w0)
tf.summary.histogram('Input_Bias', b0)
tf.summary.histogram('Hidden_Weights', w1)
tf.summary.histogram('Hidden_Bias', b1)
tf.summary.scalar('Training_Loss', train_loss)
tf.summary.scalar('Testing_Loss', test_loss)
for image in images_list:
    var_image = image[0]
    var = image[1]
    var_name = var.name.replace(':', '_')
    tf.summary.histogram('image_' + var_name, var_image)
    tf.summary.histogram(var_name, var)
summary_op = tf.summary.merge_all()
summary_folder = os.path.join(FLAGS.DEFAULT_OUT, 'MLPBaseline' + '_log', 'SGD_event')
if not os.path.exists(summary_folder):
    os.makedirs(summary_folder)
    

ValueError: Tried to convert 'values' to a tensor and failed. Error: None values not supported.

In [11]:
# Call a session
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
# initialise the graph
init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
sess.run(init_op)

# configure where to write the summary file
summary_writer = tf.summary.FileWriter(summary_folder, sess.graph)
# reset the graph
tf.reset_default_graph()

# must run initializer for iterator first that feeds in dataset
sess.run(iterator.initializer, feed_dict={features_placeholder: train_images_raw, labels_placeholder: train_labels_raw})

# run the session
start_time = time.time()
max_iter = 400
for step in range(max_iter):
    loss_value, _, global_step_value = sess.run([train_loss, opt_op, global_step])
    
    # check for NAN outcome
    assert not np.isnan(loss_value),\
        'Model diverged with loss = {} at step {}'.format(loss_value, global_step_value)
    
    # add summary and print loss every query step
    if global_step_value % 20 == 1 or global_step_value == max_iter:
        test_loss_value, accuracy = sess.run([test_loss, acc])
        if summary_op is not None:
                summary_str = sess.run(summary_op)
                summary_writer.add_summary(summary_str, global_step=global_step_value)
        print('Step {}: training loss is {:.4f}; test loss is {:.4f}; accuracy is {:.4f}%'.format(global_step_value, loss_value, test_loss_value, accuracy))

duration = time.time() - start_time        
print('Training for {} steps complete in {:.2f} seconds with a final accuracy of {:.2f}% '.format(max_iter, duration, accuracy))
sess.close()

Step 1: training loss is 169.9483; test loss is 43051.8672; accuracy is 83.2440%
Step 21: training loss is 7.8894; test loss is 2015.1200; accuracy is 87.4390%
Step 41: training loss is 6.9965; test loss is 1433.8942; accuracy is 89.0020%
Step 61: training loss is 4.7446; test loss is 1132.1034; accuracy is 89.8942%
Step 81: training loss is 4.2114; test loss is 922.9583; accuracy is 90.4410%
Step 101: training loss is 2.8853; test loss is 785.5249; accuracy is 90.7920%
Step 121: training loss is 2.5037; test loss is 689.2629; accuracy is 91.0341%
Step 141: training loss is 3.2007; test loss is 602.6517; accuracy is 91.2094%
Step 161: training loss is 2.2539; test loss is 559.1585; accuracy is 91.3309%
Step 181: training loss is 1.8680; test loss is 496.3517; accuracy is 91.4175%
Step 201: training loss is 1.7224; test loss is 466.9388; accuracy is 91.4848%
Step 221: training loss is 1.4040; test loss is 436.5746; accuracy is 91.5285%
Step 241: training loss is 1.4024; test loss is 410

In [34]:
# Data Visualisation

from tensorflow.contrib.tensorboard.plugins import projector
import os.path, os, warnings, imageio

# prepare folder
folder  = 'DataVisual'
filename = 'mnist'

summary_folder = os.path.join(FLAGS.DEFAULT_OUT, folder)

if not os.path.exists(summary_folder):
    os.makedirs(summary_folder)

embedding_path = os.path.join(summary_folder, filename + '_embedding.ckpt')
sprite_path = os.path.join(summary_folder, filename + '.png')
label_path = os.path.join(summary_folder, filename + '_label.tsv')    

# prepare data, sprite images and files
embedding_data = train_vis[0:400]   #np.reshape(train_vis, (batch_size, -1))
images = train_vis[0:400]  # shape [400, 28, 28]
image_size = train_vis.shape[1:]  # [28, 28]
labels = labels_vis[0:400]

# write label to file
if os.path.isfile(label_path):
    warnings.warn(
        'Label file {} already exists, thus this step is ignored.'.format(label_path))
else:
    metadata_file = open(label_path, 'w')
    metadata_file.write('Name\tClass\n')
    for index, label in enumerate(labels):
            metadata_file.write('%06d\t%s\n' % (index, str(label)))
    metadata_file.close()

# write images to sprite
if os.path.isfile(sprite_path):
    warnings.warn(
        'Sprite file {} already exists, thus this step is ignored.'.format(sprite_path))
else:
    # extend image shapes to [batch size, height, width, 3]
    if len(images.shape) == 3:  # if dimension of image is 3, extend it to 4
        images = np.tile(images[..., np.newaxis], (1, 1, 1, 3))
        print('Shape of images has been changed to {}'.format(images.shape))
    if images.shape[3] == 1:  # if last dimension is 1, extend it to 3
        images = np.tile(images, (1, 1, 1, 3))
        print('Shape of images has been changed to {}'.format(images.shape))
        
# invert images for mnist
images = 1 - images
    
# Tile the individual thumbnails into an image
mesh_num = (20, 20)
new_shape = mesh_num + images.shape[1:]  # (20, 20, 28, 28, 3)
images = images.reshape(new_shape).transpose((0, 2, 1, 3, 4))
print('Shape of images has been changed to {}'.format(images.shape))
images = images.reshape((mesh_num[0] * images.shape[1], mesh_num[1] * images.shape[3]) + images.shape[4:])
print('Shape of images has been changed to {}'.format(images.shape))
images = (images * 255).astype(np.uint8)
# save images to file
imageio.imwrite(sprite_path, images)

# write data to ckpt
if os.path.isfile(embedding_path):
    warnings.warn(
        'Embedding file {} already exists, thus this step is ignored.'.format(embedding_path))
else:
    # register a session
    sess = tf.Session()
    # prepare a embedding variable
    # note this must be a variable, not a tensor/constant
    embedding_var = tf.Variable(embedding_data, name='em_data')
    sess.run(embedding_var.initializer)
    # configure the embedding projector
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_var.name
    # add metadata (label) to embedding
    if label_path is not None:
        embedding.metadata_path = label_path
    # add sprite image to embedding
    if sprite_path is not None:
        embedding.sprite.image_path = sprite_path
        embedding.sprite.single_image_dim.extend(image_size)
    # finalize embedding setting
    embedding_writer = tf.summary.FileWriter(summary_folder)
    projector.visualize_embeddings(embedding_writer, config)
    embedding_saver = tf.train.Saver([embedding_var], max_to_keep=1)
    embedding_saver.save(sess, embedding_path)
    # close all
    sess.close()

Shape of images has been changed to (400, 28, 28, 3)
Shape of images has been changed to (20, 28, 20, 28, 3)
Shape of images has been changed to (560, 560, 3)
